# **Drilling Fluids Technology**

In [7]:
import pandas as pd

In [8]:
df = pd.read_excel("Clean Data.xlsx", sheet_name = "Water")
df.head()

,FLOWLINE T. (F),MD(FT),TVD (FT),INC. (DEG),AZI (DEG),DEPTH,MW (ppg),FUNNEL VISC. (SEC/QT),T. FOR PV (F),PV (cP),...,WATER ACTIVITY (Aw),SND CONTENT,MBT CAPACITY lb/bbl),PH,MUD ALKALINITY (pm)(ml),FILTRATE ALKALINITY(pf)(ml),FILTRATE ALKALINITY(Mf)(ml),CHLORIDES (mg/L),K +(mg/L),KCL %
0,NaN,3499.0,3499.0,5.15,119.87,3413.0,10.6,62,NaN,19,...,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0
1,NaN,NaN,NaN,NaN,NaN,3413.0,10.6,62,NaN,19,...,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0
2,NaN,3815.0,3815.0,5.15,119.87,3815.0,10.6,61,NaN,19,...,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0
3,NaN,NaN,NaN,NaN,NaN,3815.0,10.6,62,NaN,18,...,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0
4,NaN,3815.0,3815.0,5.15,119.87,3815.0,10.6,61,NaN,19,...,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0


In [10]:
# df_oil = pd.read_excel("Clean Data.xlsx", sheet_name = "oil")
# df_oil.head()

## **Parameters**

Plastc vscosty = 10cp

Yeld pont = 22 lb/100 ft2

Mud weght = 9.4 -10.6 ppg

hydrostatc pressure = 0.052 * MW * TVD

ECD =


In [11]:
''' PV (Plastic Viscosity) = θ₆₀₀ - θ₃₀₀ = 60 - 41 = 19 cP
 YP (Yield Point) = θ₃₀₀ - PV = 41 - 19 = 22 lb/100ft²

 From your viscometer readings in columns L and M (e.g., "60/41/33" = 600/300/200 rpm readings): '''

' PV (Plastic Viscosity) = θ₆₀₀ - θ₃₀₀ = 60 - 41 = 19 cP\nYP (Yield Point) = θ₃₀₀ - PV = 41 - 19 = 22 lb/100ft²\n\nFrom your viscometer readings in columns L and M (e.g., "60/41/33" = 600/300/200 rpm readings): '

In [19]:
import numpy as np
import pandas as pd
from typing import Tuple, Optional

class DrillingFluidsCalculator:
    """Calculator for analyzing drilling fluid parameters."""

    def __init__(self, dataframe: pd.DataFrame):
        """Initialize calculator with drilling data."""
        self.df = dataframe.copy()
        self._validate_columns()

    def _validate_columns(self):
        """Check for required columns and handle missing ones."""
        required_columns = [
            '600/300/200', 'MW (ppg)', 'TVD (FT)', 'MD(FT)',
            'API FILTRATE (ml/30min)', 'API CAKE THICKNESS (1/32in)'
        ]

        missing_cols = [col for col in required_columns if col not in self.df.columns]
        if missing_cols:
            print(f"Warning: Missing columns: {missing_cols}")

    def calculate_rheology(self) -> pd.DataFrame:
        """Calculate PV and YP from viscometer readings."""
        print("\n" + "="*50)
        print("A. PLASTIC VISCOSITY (PV) & YIELD POINT (YP)")
        print("="*50)

        def _parse_rheology_readings(value) -> Tuple[Optional[float], Optional[float]]:
            """Parse and calculate PV and YP from viscometer readings."""
            if pd.isna(value):
                return np.nan, np.nan

            try:
                # Handle both string and numeric inputs
                readings_str = str(value).strip()
                if '/' in readings_str:
                    readings = [int(x.strip()) for x in readings_str.split('/')[:2]]
                else:
                    # If no slash, assume it's already theta_600
                    theta_600 = float(readings_str)
                    return theta_600, np.nan

                theta_600, theta_300 = readings[0], readings[1]
                pv = theta_600 - theta_300
                yp = theta_300 - pv

                # Validate results
                if pv < 0 or yp < 0:
                    return np.nan, np.nan

                return pv, yp

            except (ValueError, IndexError, AttributeError) as e:
                print(f"Warning: Could not parse rheology reading: {value}")
                return np.nan, np.nan

        # Apply calculations
        results = self.df['600/300/200'].apply(_parse_rheology_readings)
        self.df[['Calculated PV (cP)', 'Calculated YP (lbf/100ft2)']] = pd.DataFrame(
            results.tolist(), index=self.df.index
        )

        # Display comparison
        display_cols = ['600/300/200', 'PV (cP)', 'Calculated PV (cP)',
                       'YP (lbf/100ft2)', 'Calculated YP (lbf/100ft2)']

        # Only show columns that exist
        display_cols = [col for col in display_cols if col in self.df.columns]

        print(f"Rheology calculations completed. Samples: {len(self.df)}")
        print(f"Successfully calculated: {self.df['Calculated PV (cP)'].notna().sum()} rows")

        if display_cols:
            display(self.df[display_cols].head(10))

        return self.df

    def calculate_hydrostatic_pressure(self) -> pd.DataFrame:
        """Calculate hydrostatic pressure from TVD and mud weight."""
        print("\n" + "="*50)
        print("C. HYDROSTATIC PRESSURE CALCULATION")
        print("="*50)

        # Convert to numeric with error handling
        self.df['TVD (FT)'] = pd.to_numeric(self.df['TVD (FT)'], errors='coerce')
        self.df['MW (ppg)'] = pd.to_numeric(self.df['MW (ppg)'], errors='coerce')

        # Calculate hydrostatic pressure
        self.df['Calculated HP (psi)'] = 0.052 * self.df['MW (ppg)'] * self.df['TVD (FT)']

        # Statistics
        valid_count = self.df['Calculated HP (psi)'].notna().sum()
        avg_hp = self.df['Calculated HP (psi)'].mean()

        print(f"Valid calculations: {valid_count}/{len(self.df)} rows")
        print(f"Average HP: {avg_hp:.1f} psi")
        print(f"MW range: {self.df['MW (ppg)'].min():.1f} - {self.df['MW (ppg)'].max():.1f} ppg")
        print(f"TVD range: {self.df['TVD (FT)'].min():.0f} - {self.df['TVD (FT)'].max():.0f} ft")

        display_cols = ['MD(FT)', 'TVD (FT)', 'MW (ppg)', 'Calculated HP (psi)']
        display_cols = [col for col in display_cols if col in self.df.columns]

        if display_cols:
            display(self.df[display_cols].head(10))

        return self.df

    def analyze_fluid_loss(self):
        """Analyze fluid loss and wall cake quality."""
        print("\n" + "="*50)
        print("D. FLUID LOSS & WALL CAKE QUALITY")
        print("="*50)

        # API Fluid Loss (Water-based mud)
        print("\nAPI Filtrate (Water-based mud):")
        api_cols = ['API FILTRATE (ml/30min)', 'API CAKE THICKNESS (1/32in)']
        api_cols = [col for col in api_cols if col in self.df.columns]

        if api_cols:
            display(self.df[api_cols].head(10))

            # Add quality assessment
            if 'API FILTRATE (ml/30min)' in self.df.columns:
                filtrate = pd.to_numeric(self.df['API FILTRATE (ml/30min)'], errors='coerce')
                high_loss = (filtrate > 10).sum()
                print(f"High fluid loss (>10 ml/30min): {high_loss} samples")

        # HTHP Fluid Loss (Oil-based mud)
        print("\nHTHP Filtrate (Oil-based mud):")
        hthp_cols = ['HTHP FILTRATE (ml/30MIN)', 'HTHP CAKE THICKNESS(1/32IN)']
        hthp_cols = [col for col in hthp_cols if col in self.df.columns]

        if hthp_cols:
            display(self.df[hthp_cols].head(10))

    def analyze_solids_control(self):
        """Analyze solids control parameters."""
        print("\n" + "="*50)
        print("E. SOLIDS CONTROL ANALYSIS")
        print("="*50)

        solids_cols = [
            'OIL/WATER RATIO', 'CHLORIDES (mg/L)',
            'SALT CONTENT WATER PHASE (%)', 'BRINE DENSITY(ppg)'
        ]

        available_cols = [col for col in solids_cols if col in self.df.columns]

        if available_cols:
            display(self.df[available_cols].head(10))

            # Calculate statistics for numeric columns
            for col in available_cols:
                if col in ['CHLORIDES (mg/L)', 'SALT CONTENT WATER PHASE (%)', 'BRINE DENSITY(ppg)']:
                    numeric_data = pd.to_numeric(self.df[col], errors='coerce')
                    if numeric_data.notna().any():
                        print(f"\n{col}:")
                        print(f"  Range: {numeric_data.min():.1f} - {numeric_data.max():.1f}")
                        print(f"  Mean: {numeric_data.mean():.1f}")
        else:
            print("No solids control data available.")

    def generate_summary(self):
        """Generate a summary report of all calculations."""
        print("\n" + "="*50)
        print("DRILLING FLUIDS ANALYSIS SUMMARY")
        print("="*50)

        summary_data = {
            'Parameter': ['Total Samples', 'MW Range (ppg)', 'TVD Range (ft)',
                         'Avg HP (psi)', 'PV Calculated', 'YP Calculated'],
            'Value': []
        }

        # Calculate summary values
        total_samples = len(self.df)
        mw_range = f"{self.df['MW (ppg)'].min():.1f}-{self.df['MW (ppg)'].max():.1f}"
        tvd_range = f"{self.df['TVD (FT)'].min():.0f}-{self.df['TVD (FT)'].max():.0f}"
        avg_hp = f"{self.df['Calculated HP (psi)'].mean():.0f}"
        pv_calc = f"{self.df['Calculated PV (cP)'].notna().sum()}/{total_samples}"
        yp_calc = f"{self.df['Calculated YP (lbf/100ft2)'].notna().sum()}/{total_samples}"

        summary_data['Value'] = [total_samples, mw_range, tvd_range, avg_hp, pv_calc, yp_calc]

        summary_df = pd.DataFrame(summary_data)
        display(summary_df)

        return self.df

    def run_full_analysis(self):
        """Execute complete drilling fluids analysis."""
        print("INITIATING DRILLING FLUIDS ANALYSIS")
        print("="*50)

        # Run all analyses
        self.calculate_rheology()
        self.calculate_hydrostatic_pressure()
        self.analyze_fluid_loss()
        self.analyze_solids_control()
        self.generate_summary()

        print("\n" + "="*50)
        print("ANALYSIS COMPLETE")
        print("="*50)

        return self.df


# Usage function
def analyze_drilling_fluids(dataframe: pd.DataFrame, return_dataframe: bool = False):
    """
    Main function to analyze drilling fluids data.

    Parameters:
    -----------
    dataframe : pd.DataFrame
        Input dataframe containing drilling fluids data
    return_dataframe : bool
        If True, returns the analyzed dataframe

    Returns:
    --------
    pd.DataFrame or None
        Analyzed dataframe if return_dataframe=True
    """
    try:
        # Create calculator instance
        calculator = DrillingFluidsCalculator(dataframe)

        # Run analysis
        result_df = calculator.run_full_analysis()

        if return_dataframe:
            return result_df

    except Exception as e:
        print(f"Error during analysis: {str(e)}")
        raise


# Example usage
if __name__ == "__main__":
    # Assuming df is your dataframe
    # analyze_drilling_fluids(df)

    # Or to get the analyzed dataframe back:
    # analyzed_df = analyze_drilling_fluids(df, return_dataframe=True)
    pass

Performing Critical Drilling Fluids Calculations:

--- A. Plastic Viscosity (PV) & Yield Point (YP) ---


,600/300/200,PV (cP),Calculated PV (cP),YP (lbf/100ft2),Calculated YP (lbf/100ft2)
0,NaN,19,NaN,22,NaN
1,NaN,19,NaN,22,NaN
2,60/41/33,19,19.0,22,22.0
3,59/41/32,18,18.0,23,23.0
4,60/41/33,19,19.0,22,22.0



--- B. Mud Weight (MW) ---
Mud Weight (ppg) is directly given in column 'MW (ppg)'.


,MW (ppg)
0,10.6
1,10.6
2,10.6
3,10.6
4,10.6



--- C. Hydrostatic Pressure ---


...

,API FILTRATE (ml/30min),API CAKE THICKNESS (1/32in)
0,4.8,1.0
1,4.8,1.0
2,4.8,1.0
3,4.8,1.0
4,4.8,1.0


HTHP Filtrate and Cake Thickness (Oil-based mud, if available/relevant to this sheet): 


,HTHP FILTRATE (ml/30MIN),HTHP CAKE THICKNESS(1/32IN)
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN



--- E. Solids Control Analysis ---
Oil/Water Ratio, Chlorides, and Brine Density:


,OIL/WATER RATIO,CHLORIDES (mg/L),SALT CONTENT WATER PHASE (%),BRINE DENSITY(ppg)
0,NaN,48000.0,NaN,NaN
1,NaN,48000.0,NaN,NaN
2,NaN,48000.0,NaN,NaN
3,NaN,48000.0,NaN,NaN
4,NaN,48000.0,NaN,NaN


In [17]:
pd.set_option('display.max_columns', None)
display(df.head(1))

,FLOWLINE T. (F),MD(FT),TVD (FT),INC. (DEG),AZI (DEG),DEPTH,MW (ppg),FUNNEL VISC. (SEC/QT),T. FOR PV (F),PV (cP),YP (lbf/100ft2),600/300/200,100/6/3,GEL STR. (10SEC) (lbf/100ft2),GEL STR. (10 min) (lbf/100ft2),T. FOR HTHP (F),HTHP FILTRATE (ml/30MIN),HTHP CAKE THICKNESS(1/32IN),API FILTRATE (ml/30min),API CAKE THICKNESS (1/32in),SOLIDS (%),OIL (%),WATER (%),OIL/WATER RATIO,ALKALINITY MUD (pom)(cc/cc),EXCESS LIME (lb/bbl),CHLORIDE WHOLE MUD (mg/L),SOLIDS ADJUSTED FOR SALT(%),SALT CONTENT WATER PHASE (%),WPS(ppm),CaCl2 wt (%),CaCl2 wt (mg/L),BRINE DENSITY(ppg),ELECTRICAL STABILITY (VOLT),WATER ACTIVITY (Aw),SND CONTENT,MBT CAPACITY lb/bbl),PH,MUD ALKALINITY (pm)(ml),FILTRATE ALKALINITY(pf)(ml),FILTRATE ALKALINITY(Mf)(ml),CHLORIDES (mg/L),K +(mg/L),KCL %
0,NaN,3499.0,3499.0,5.15,119.87,3413.0,10.6,62,NaN,19,22,NaN,NaN,13,37,NaN,NaN,NaN,4.8,1.0,5,4,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.0,9.0,0.4,0.2,0.3,48000.0,48000.0,10.0


In [18]:
data =df[['MD(FT)','TVD (FT)','MW (ppg)','T. FOR PV (F)','PV (cP)','YP (lbf/100ft2)','GEL STR. (10SEC) (lbf/100ft2)','GEL STR. (10 min) (lbf/100ft2)']]
data.head()

,MD(FT),TVD (FT),MW (ppg),T. FOR PV (F),PV (cP),YP (lbf/100ft2),GEL STR. (10SEC) (lbf/100ft2),GEL STR. (10 min) (lbf/100ft2)
0,3499.0,3499.0,10.6,NaN,19,22,13,37
1,NaN,NaN,10.6,NaN,19,22,13,37
2,3815.0,3815.0,10.6,NaN,19,22,13,37
3,NaN,NaN,10.6,NaN,18,23,13,37
4,3815.0,3815.0,10.6,NaN,19,22,13,37
